The script aims to solve the problem: `Blocking for next trial...` when resuming the ray.tune

In [1]:
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray import tune
import os
import json
import numpy as np
import pdb

In [2]:
from ray.tune.utils.serialization import TuneFunctionEncoder,TuneFunctionDecoder

In [3]:
restore_folder=os.path.join(os.getcwd(),'data/ray_results/20220614_trial1')

In [4]:
bayesopt = BayesOptSearch(metric="score",
                          mode="min",
                          random_search_steps=150)
bayesopt.restore(os.path.join(restore_folder, 'searcher-state-2022-06-17_14-36-27.pkl'))
bayesopt._total_random_search_trials

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianProcessRegressor from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


150

In [5]:
len(bayesopt._buffered_trial_results)

147

In [6]:
bayesopt._total_random_search_trials=147
bayesopt.save(os.path.join(restore_folder, 'searcher-state-2022-06-17_15-00-00.pkl'))

In [7]:
bayesopt._buffered_trial_results[0][0]['c_ii']

9.909729556485983

In [8]:
with open(os.path.join(restore_folder,'experiment_state-2022-06-17_14-36-27.json')) as f:
    ckp=json.load(f)
checkpoint=ckp['checkpoints']
f.close()

In [13]:
checkpoint[0].find('\n    "c_ii": ')

187

In [19]:
float(checkpoint[0][200:217])

7.666323431412191

In [24]:
checkpoint[0][187+13:187+13+17]

'7.666323431412191'

In [9]:
useless=[]
cts = []
for i in range(len(checkpoint)):
    c_ii_s = checkpoint[i].find('\n    "c_ii": ')
    c_ii = float(checkpoint[i][c_ii_s+13:c_ii_s+13+17])
    findone = False
    ct = 0
    for j in range(len(bayesopt._buffered_trial_results)):
        if bayesopt._buffered_trial_results[j][0]['c_ii']== c_ii:
            findone = True
            ct+=1
    cts.append(ct)
    if findone==False:
        useless.append(i)

In [10]:
useless

[17, 39, 47, 51, 98, 104, 131]

In [53]:
np.where(np.array(cts)==2)[0].shape

(0,)

In [67]:
useless2=[]
cts2 = []
for i in range(len(bayesopt._buffered_trial_results)):
    baye_cii = bayesopt._buffered_trial_results[i][0]['c_ii']
    findone = False
    ct = 0
    for j in range(len(checkpoint)):
        c_ii_s = checkpoint[j].find('\n    "c_ii": ')
        c_ii = float(checkpoint[j][c_ii_s+13:c_ii_s+13+17])
        if baye_cii== c_ii:
            findone = True
            ct+=1
    cts2.append(ct)
    if findone==False:
        useless2.append(i)

In [68]:
useless2

[71, 72, 73, 129]

In [72]:
dir(bayesopt)

['CKPT_FILE_TMPL',
 'FINISHED',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_analysis',
 '_buffered_trial_results',
 '_config_counter',
 '_live_trial_mapping',
 '_metric',
 '_metric_op',
 '_mode',
 '_patience',
 '_points_to_evaluate',
 '_random_state',
 '_register_result',
 '_setup_optimizer',
 '_skip_duplicate',
 '_space',
 '_total_random_search_trials',
 '_verbose',
 'add_evaluated_point',
 'add_evaluated_trials',
 'convert_search_space',
 'get_state',
 'max_concurrent',
 'metric',
 'mode',
 'on_trial_complete',
 'on_trial_result',
 'optimizer',
 'random_search_trials',
 'register_analysis',
 'repeat_float_precision',
 'restore',
 'restore_from_dir',
 'save',
 '

In [127]:
bayesopt._total_random_search_trials

147

In [96]:
print(bayesopt._analysis)

None


In [74]:
results = np.array(bayesopt._buffered_trial_results)

In [77]:
results[72][0]['c_ii']

7.1120445662476435

In [91]:
new_results = np.delete(results, useless2,axis=0)

In [93]:
c_iis = np.array([new_results[i][0]['c_ii'] for i in range(len(new_results))])

In [94]:
np.where(c_iis==results[72][0]['c_ii'])

(array([], dtype=int64),)

In [125]:
bopt = bayesopt
bopt._buffered_trial_results = new_results

In [128]:
bopt._total_random_search_trials=143

In [129]:
bopt.save(os.path.join(restore_folder, 'searcher-state-2022-06-17_15-00-00.pkl'))

In [97]:
def array2tuple(x):
    return (x[0],x[1])

In [106]:
new_buffer = list(map(array2tuple,new_results))
len(new_buffer)

143

In [11]:
ckpt_array = np.array(checkpoint)
new_ckpt_array = np.delete(ckpt_array,useless)

In [12]:
new_ckpt=list(map(str,new_ckpt_array))

In [13]:
len(new_ckpt)

143

In [14]:
fp = '/mnt/user/drive/My Libraries/tutorials&explorations/data/ray_results/20220614_trial1/experiment_state-2022-06-17_15-00-00.json'

In [34]:
ckp['checkpoints'] = new_ckpt

In [35]:
ckp['runner_data']['checkpoint_file'] = fp

In [36]:
ckp['stats']

{'start_time': 1655469387.466305, 'timestamp': 1655547700.0619483}

In [37]:
with open(fp, "w") as f:
    json.dump(ckp, f, indent=2, cls=TuneFunctionEncoder)
f.close()

In [2]:
from ray.tune.suggest.suggestion import ConcurrencyLimiter, Searcher
from ray.tune.suggest import SearchGenerator
search_alg = BayesOptSearch(metric="score",
                          mode="min",
                          random_search_steps=150)
max_concurrent_trials = 3
search_alg = ConcurrencyLimiter(search_alg, max_concurrent=max_concurrent_trials)

In [3]:
if isinstance(search_alg, Searcher):
        search_alg = SearchGenerator(search_alg)

In [6]:
dirpath = '/mnt/user/drive/My Libraries/tutorials&explorations/data/ray_results/20220614_trial1'
search_alg.restore_from_dir(dirpath)

In [10]:
CKPT_FILE_TMPL = "search_gen_state-{}.json"
search_alg_state = tune.utils.util.load_newest_checkpoint(
            dirpath, CKPT_FILE_TMPL.format("*")
        )

In [23]:
search_alg_state['counter']=143

In [25]:
tune.utils.util.atomic_save(state=search_alg_state,checkpoint_dir=dirpath,file_name='search_gen_state-2022-06-17_15-00-00.json',tmp_file_name='search_gen_state-2022-06-17_15-00-00.json')

In [38]:
with open(fp) as f:
    runner_state = json.load(f, cls=TuneFunctionDecoder)
    f.close()

In [39]:
len(runner_state['checkpoints'])

143